Building an interactive web app for technical analysis
using Streamlit

Streamlit is an open source framework (and a company
under the same name, similarly to Plotly) that allows us to build interactive web apps using only Python, all within minutes. Below you can find the highlights of Streamlit:

• It is easy to learn and can generate results very quickly

• It is Python only; no frontend experience is required

• It allows us to focus purely on the data/ML sides of the app

• We can use Streamlit’s hosting services for our apps

In [1]:
# imports
import yfinance as yf
import streamlit as st
import datetime 
import pandas as pd
import cufflinks as cf
from plotly.offline import iplot

In [2]:
## set offline mode for cufflinks
cf.go_offline()

In [7]:
#Define a function for downloading a list of IBEX 35 constituents from Wikipedia:
# Define the custom hash function
def my_hash_func(obj):
    return hash(str(obj))

# Define your function with caching and the custom hash function
@st.cache_data(hash_funcs={type(lambda: None): my_hash_func})
def get_IBEX35_components():
    df = pd.read_html("https://www.dividendmax.com/market-index-constituents/ibex-35")
    df = df[0]
    print(df.columns)  # Print the column names to identify the correct ones
    print(df.head())   # Print the first few rows to inspect the DataFrame structure
    tickers = df["Ticker"].to_list()
    tickers_companies_dict = dict(
        zip(df["Ticker"], df["Company"])
    )
    return tickers, tickers_companies_dict
    


2024-05-29 12:36:15.183 No runtime found, using MemoryCacheStorageManager


In [8]:
@st.cache_data
def load_data(symbol, start, end):
    return yf.download(symbol, start, end)


2024-05-29 12:36:19.348 No runtime found, using MemoryCacheStorageManager


In [9]:

@st.cache_data
def convert_df_to_csv(df):
    return df.to_csv().encode("utf-8")

2024-05-29 12:36:24.981 No runtime found, using MemoryCacheStorageManager


In [10]:
## inputs for downloading data
st.sidebar.header("Stock Parameters")

available_tickers, tickers_companies_dict = get_IBEX35_components()
ticker = st.sidebar.selectbox(
    "Ticker", 
    available_tickers, 
    format_func=tickers_companies_dict.get
)
start_date = st.sidebar.date_input(
    "Start date", 
    datetime.date(2019, 1, 1)
)
end_date = st.sidebar.date_input(
    "End date", 
    datetime.date.today()
)

if start_date > end_date:
    st.sidebar.error("The end date must fall after the start date")

2024-05-29 12:36:26.666 No runtime found, using MemoryCacheStorageManager


Index(['Company', 'Ticker', 'Unnamed: 2', 'Exchange', 'Sector', 'Market Cap'], dtype='object')
                                             Company Ticker Unnamed: 2  \
0                                            Acciona    ANA         🇪🇸   
1                                           Acerinox    ACX         🇪🇸   
2  ACS, Actividades de Construccion Y Servicios, ...    ACS         🇪🇸   
3                                   Aena S.M.E. S.A.   AENA         🇪🇸   
4                              Amadeus IT Group S.A.    AMS         🇪🇸   

                Exchange                        Sector Market Cap  
0  Madrid Stock Exchange     Commercial Transportation     €6.6bn  
1  Madrid Stock Exchange             Industrial Metals     €2.5bn  
2  Madrid Stock Exchange      Construction & Materials    €11.2bn  
3  Madrid Stock Exchange              Travel & Leisure    €26.8bn  
4  Madrid Stock Exchange  Software & Computer Services    €28.8bn  


In [11]:
## inputs for technical analysis
st.sidebar.header("Technical Analysis Parameters")

volume_flag = st.sidebar.checkbox(label="Add volume")

exp_sma = st.sidebar.expander("SMA")
sma_flag = exp_sma.checkbox(label="Add SMA")
sma_periods= exp_sma.number_input(
    label="SMA Periods", 
    min_value=1, 
    max_value=50, 
    value=20, 
    step=1
)

exp_bb = st.sidebar.expander("Bollinger Bands")
bb_flag = exp_bb.checkbox(label="Add Bollinger Bands")
bb_periods= exp_bb.number_input(label="BB Periods", 
                                min_value=1, max_value=50, 
                                value=20, step=1)
bb_std= exp_bb.number_input(label="# of standard deviations", 
                            min_value=1, max_value=4, 
                            value=2, step=1)

exp_rsi = st.sidebar.expander("Relative Strength Index")
rsi_flag = exp_rsi.checkbox(label="Add RSI")
rsi_periods= exp_rsi.number_input(
    label="RSI Periods", 
    min_value=1, 
    max_value=50, 
    value=20, 
    step=1
)
rsi_upper= exp_rsi.number_input(label="RSI Upper", 
                                min_value=50, 
                                max_value=90, value=70, 
                                step=1)
rsi_lower= exp_rsi.number_input(label="RSI Lower", 
                                min_value=10, 
                                max_value=50, value=30, 
                                step=1)


In [12]:
# main body

st.title("A simple web app for technical analysis")
st.write("""
### User manual
* you can select any of the companies that is a component of the S&P index
* you can select the time period of your interest
* you can download the selected data as a CSV file
* you can add the following Technical Indicators to the plot: Simple Moving 
Average, Bollinger Bands, Relative Strength Index
* you can experiment with different parameters of the indicators
""")

df = load_data(ticker, start_date, end_date)

2024-05-29 12:36:30.732 No runtime found, using MemoryCacheStorageManager
[*********************100%%**********************]  1 of 1 completed


In [13]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,0.0038,0.0038,0.0036,0.0038,0.0038,285007
2019-01-03,0.0038,0.0038,0.0038,0.0038,0.0038,0
2019-01-04,0.0038,0.0038,0.0038,0.0038,0.0038,0
2019-01-07,0.0038,0.0038,0.0036,0.0038,0.0038,1714985
2019-01-08,0.0038,0.0038,0.0038,0.0038,0.0038,0
...,...,...,...,...,...,...
2022-02-24,0.8250,0.8250,0.8250,0.8250,0.8250,0
2022-02-25,0.8250,0.8250,0.8250,0.8250,0.8250,0
2022-02-28,0.8250,0.8250,0.8250,0.8250,0.8250,0


In [14]:
## data preview part
data_exp = st.expander("Preview data")
available_cols = df.columns.tolist()
columns_to_show = data_exp.multiselect(
    "Columns", 
    available_cols, 
    default=available_cols
)
data_exp.dataframe(df[columns_to_show])

csv_file = convert_df_to_csv(df[columns_to_show])
data_exp.download_button(
    label="Download selected as CSV",
    data=csv_file,
    file_name=f"{ticker}_stock_prices.csv",
    mime="text/csv",
)

2024-05-29 12:37:28.480 No runtime found, using MemoryCacheStorageManager


False

In [15]:
## technical analysis plot
title_str = f"{tickers_companies_dict[ticker]}'s stock price"
qf = cf.QuantFig(df, title=title_str)
if volume_flag:
    qf.add_volume()
if sma_flag:
    qf.add_sma(periods=sma_periods)
if bb_flag:
    qf.add_bollinger_bands(periods=bb_periods,
                           boll_std=bb_std)
if rsi_flag:
    qf.add_rsi(periods=rsi_periods,
               rsi_upper=rsi_upper,
               rsi_lower=rsi_lower,
               showbands=True)

fig = qf.iplot(asFigure=True)
st.plotly_chart(fig)

DeltaGenerator()